
# Multivariate Functional Principal Components Analysis

This notebook shows how to perform an multivariate functional principal
components analysis on an example dataset.


In [41]:
# Author: Zara Waheed <zara95@bu.edu>
# License: MIT

import matplotlib.pyplot as plt
import pandas as pd

from FDApy.representation.functional_data import MultivariateFunctionalData
from FDApy.preprocessing.dim_reduction.fpca import MFPCA
from FDApy.visualization.plot import plot
from FDApy.misc.loader import read_csv
import numpy as np

In [16]:
def wf(x):
    return '/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Raw/' + x

Load the data as DenseFunctionalData.



In [17]:
V_GRF_stance_N = read_csv(wf('V_GRF_stance_N.csv'))
ML_GRF_stance_N = read_csv(wf('ML_GRF_stance_N.csv'))
AP_GRF_stance_N = read_csv(wf('AP_GRF_stance_N.csv'))

#ML_GRF_stance_N = ML_GRF_stance_N.reset_index()
#AP_GRF_stance_N = AP_GRF_stance_N.reset_index()

In [18]:
# Create multivariate functional data

GRF = MultivariateFunctionalData([ML_GRF_stance_N, AP_GRF_stance_N, V_GRF_stance_N])

Perform a multivariate functional PCA and explore the results.



In [ ]:
# Perform multivariate FPCA

mfpca = MFPCA(n_components = [4, 3, 3])
# n_components is number of components to keep for each functions in data

mfpca.fit(GRF, method='NumInt')
# other method = PACE 

# Plot the results of the FPCA (eigenfunctions)
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,6))
_ = plot(mfpca.basis[0], ax=ax1)
_.set_ylabel('ML_GRF')
_ = plot(mfpca.basis[1], ax=ax2)
_.set_ylabel('AP_GRF')
_ = plot(mfpca.basis[2], ax=ax3)
_.set_ylabel('V_GRF')
_.set_xlabel('Time')
ax1.set_title('MFPCA plot with 10 Principal Components', fontstyle='italic')

fig.savefig('/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Processed/Plots/MFPCA_combined.png', dpi=300)


In [ ]:
# mfpca.basis

Compute the scores of the dailyTemp data into the eigenfunctions basis using
numerical integration.



In [ ]:
# Compute the scores
GRF_proj = mfpca.transform(GRF)

# Plot the projection of the data onto the eigenfunctions
fig = pd.plotting.scatter_matrix(pd.DataFrame(GRF_proj), diagonal='kde', figsize=(15, 15))
plt.savefig('/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Processed/MFPCA_scatter_matrix.png', dpi=300)

Then, we can test if the reconstruction of the data is good.



In [ ]:
# Test if the reconstruction is good.
GRF_reconst = mfpca.inverse_transform(GRF_proj)

# Plot the reconstructed curves
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (15,12))
_ = plot(GRF_reconst[0], ax=ax1)
_.set_ylabel('GRF_ML')
_ = plot(GRF_reconst[1], ax=ax2)
_.set_ylabel('GRF_AP')
_ = plot(GRF_reconst[2], ax=ax3)
_.set_ylabel('GRF_V')
_.set_xlabel('Time')
ax1.set_title('MFPCA Reconstructed Data with 8 Principal Components', fontstyle='italic')
fig.savefig('/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Processed/MFPCA_reconstructed_plot.png', dpi=300)

### Calculate MSE and TSS

In [19]:
# Define function for MSE

def mse(array1,array2):
    diff = array1 - array2
    sqdiff = diff**2
    n = len(sqdiff)
    
    sum_sqdiff = sum(sqdiff)
    return sum_sqdiff/n

In [ ]:
# Define function for TSS

def tss(array1,array2):
    diff = array1 - array2
    sqdiff = diff**2
    n = len(sqdiff)
    
    sum_sqdiff = sum(sqdiff)
    return sum_sqdiff/n

In [20]:
# define number on components

n = 8

In [127]:
# run mfpca

mfpca = MFPCA(n_components = [2, 2, 2])
mfpca.fit(GRF, method='NumInt')
GRF_reconst = mfpca.inverse_transform(mfpca.transform(GRF))

In [128]:
# Create datasets

# create ML_GRF reconstructed and original data array
ML_GRF_reconst = pd.DataFrame(GRF_reconst[0].values)
ML_GRF_reconst_array = ML_GRF_reconst.values.flatten()

ML_GRF_df = pd.DataFrame(ML_GRF_stance_N.values)
ML_GRF_array = ML_GRF_df.values.flatten()
    
ML_mean = sum(ML_GRF_array)/len(ML_GRF_array)
ML_mean_array = np.full(shape=1569500,fill_value=ML_mean)
    
# create AP_GRF reconstructed and original data array
AP_GRF_reconst = pd.DataFrame(GRF_reconst[1].values)
AP_GRF_reconst_array = AP_GRF_reconst.values.flatten()

AP_GRF_df = pd.DataFrame(AP_GRF_stance_N.values)
AP_GRF_array = AP_GRF_df.values.flatten()
    
AP_mean = sum(AP_GRF_array)/len(AP_GRF_array)
AP_mean_array = np.full(shape=1569500,fill_value=AP_mean)
    
# create V_GRF reconstructed and original data array
V_GRF_reconst = pd.DataFrame(GRF_reconst[2].values)
V_GRF_reconst_array = V_GRF_reconst.values.flatten()

V_GRF_df = pd.DataFrame(V_GRF_stance_N.values)
V_GRF_array = V_GRF_df.values.flatten()
    
V_mean = sum(V_GRF_array)/len(V_GRF_array)
V_mean_array = np.full(shape=1569500,fill_value=V_mean)


In [129]:
# Calculate mean

ML_GRF_mean = round(mse(ML_GRF_array, ML_GRF_reconst_array), 2)
AP_GRF_mean = round(mse(AP_GRF_array, AP_GRF_reconst_array), 2)
V_GRF_mean = round(mse(V_GRF_array, V_GRF_reconst_array), 2)

# Calculate TSS

ML_TSS = round(mse(ML_GRF_array, ML_mean_array), 2)
AP_TSS = round(mse(AP_GRF_array, AP_mean_array), 2)
V_TSS = round(mse(V_GRF_array, V_mean_array), 2)
    

In [130]:
# Create initial dataset

new = pd.DataFrame({'Components' : [2], 
                    'MSE_ML': [ML_GRF_mean],
                    'MSE_AP': [AP_GRF_mean],
                    'MSE_V': [V_GRF_mean],
                    'TSS_ML': [ML_TSS],
                    'TSS_AP': [AP_TSS],
                    'TSS_V': [V_TSS]
                       })
mse_GRF = new
#mse_GRF2 = pd.concat([mse_GRF2, new])

In [124]:
#mse_GRF2['R_ML'] = round(1 - (mse_GRF2['MSE_ML']/mse_GRF2['TSS_ML']), 2)
#mse_GRF2['R_AP'] = round(1 - (mse_GRF2['MSE_AP']/mse_GRF2['TSS_AP']), 2)
#mse_GRF2['R_V'] = round(1 - (mse_GRF2['MSE_V']/mse_GRF2['TSS_V']), 2)

In [131]:
# Create a loop that runs the model, creates reconstructed datasets, calculates mse and adds it to our dataset

for i in range(4, 22, 2):
    
    # fit the model and create reconstructed dataset
    mfpca = MFPCA(n_components = [i, i, i])
    mfpca.fit(GRF, method='NumInt')
    GRF_reconst = mfpca.inverse_transform(mfpca.transform(GRF))

    # create ML_GRF reconstructed and original data array
    ML_GRF_reconst = pd.DataFrame(GRF_reconst[0].values)
    ML_GRF_reconst_array = ML_GRF_reconst.values.flatten()

    ML_GRF_df = pd.DataFrame(ML_GRF_stance_N.values)
    ML_GRF_array = ML_GRF_df.values.flatten()
    
    ML_mean = sum(ML_GRF_array)/len(ML_GRF_array)
    ML_mean_array = np.full(shape=1569500,fill_value=ML_mean)
    
    # create AP_GRF reconstructed and original data array
    AP_GRF_reconst = pd.DataFrame(GRF_reconst[1].values)
    AP_GRF_reconst_array = AP_GRF_reconst.values.flatten()

    AP_GRF_df = pd.DataFrame(AP_GRF_stance_N.values)
    AP_GRF_array = AP_GRF_df.values.flatten()
    
    AP_mean = sum(AP_GRF_array)/len(AP_GRF_array)
    AP_mean_array = np.full(shape=1569500,fill_value=AP_mean)
    
    # create V_GRF reconstructed and original data array
    V_GRF_reconst = pd.DataFrame(GRF_reconst[2].values)
    V_GRF_reconst_array = V_GRF_reconst.values.flatten()

    V_GRF_df = pd.DataFrame(V_GRF_stance_N.values)
    V_GRF_array = V_GRF_df.values.flatten()
    
    V_mean = sum(V_GRF_array)/len(V_GRF_array)
    V_mean_array = np.full(shape=1569500,fill_value=V_mean)

    # Calculate mean

    ML_GRF_mean = round(mse(ML_GRF_array, ML_GRF_reconst_array), 2)
    AP_GRF_mean = round(mse(AP_GRF_array, AP_GRF_reconst_array), 2)
    V_GRF_mean = round(mse(V_GRF_array, V_GRF_reconst_array), 2)

    # Calculate TSS

    ML_TSS = round(mse(ML_GRF_array, ML_mean_array), 2)
    AP_TSS = round(mse(AP_GRF_array, AP_mean_array), 2)
    V_TSS = round(mse(V_GRF_array, V_mean_array), 2)    
    
    # Store the values in the dataframe
    
    new = pd.DataFrame({'Components' : [i],
                          'MSE_ML': [ML_GRF_mean],
                          'MSE_AP': [AP_GRF_mean],
                          'MSE_V': [V_GRF_mean],
                          'TSS_ML': [ML_TSS],
                          'TSS_AP': [AP_TSS],
                          'TSS_V': [V_TSS]
                         })
    
    mse_GRF = pd.concat([mse_GRF, new])


/var/folders/3f/jf0vyw7d5dz_ypqzwp3mnylm0000gn/T/ipykernel_41469/1650186797.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ML_mean_array = np.full(shape=1569500,fill_value=ML_mean, dtype=np.int)
/var/folders/3f/jf0vyw7d5dz_ypqzwp3mnylm0000gn/T/ipykernel_41469/1650186797.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the preci

/var/folders/3f/jf0vyw7d5dz_ypqzwp3mnylm0000gn/T/ipykernel_41469/1650186797.py:38: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  V_mean_array = np.full(shape=1569500,fill_value=V_mean, dtype=np.int)
/var/folders/3f/jf0vyw7d5dz_ypqzwp3mnylm0000gn/T/ipykernel_41469/1650186797.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precisi

In [132]:
mse_GRF['R_ML'] = round(1 - (mse_GRF['MSE_ML']/mse_GRF['TSS_ML']), 2)
mse_GRF['R_AP'] = round(1 - (mse_GRF['MSE_AP']/mse_GRF['TSS_AP']), 2)
mse_GRF['R_V'] = round(1 - (mse_GRF['MSE_V']/mse_GRF['TSS_V']), 2)

In [134]:
# Save the dataset

mse_GRF.to_csv('/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Processed/Tables/MFPCA_mse.csv')
#mse_GRF2.to_csv('/Users/zarawaheed/Documents/BostonUniversity/MA679/Final Project/Data/Processed/Tables/MFPCA_mse_combined.csv')